1. Import and Install Dependencies


In [ ]:
%pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [ ]:

import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

2. Keypoints using MP Holistic

3. Extract Keypoint Values

4. Setup Folders for Collection

5. Collect Keypoint Values for Training and Testing

6. Preprocess Data and Create Labels and Features

7. Build and Train LSTM Neural Network

8. Make Predictions

9. Save Weights


10. Evaluation using Confusion Matrix and Accuracy


11. Test in Real Time
